In [1]:
import pandas as pd
import numpy as np
import gc
from sklearn.utils import resample
import featuretools as ft

## Set random number seed and import main table in hierarchy

In [2]:
seed=123

APPLICATION_TRAIN = pd.read_csv("application_train.csv",na_values=['XNA','XAP'])
APPLICATION_TRAIN['DAYS_EMPLOYED'].replace(365243,np.nan,inplace=True)

APPLICATION_TRAIN['AMT_GOODS_PRICE-AMT_CREDIT']=APPLICATION_TRAIN['AMT_GOODS_PRICE']-APPLICATION_TRAIN['AMT_CREDIT']

## Downsample using resample function imported above.  Not enough memory to use an oversampled dataset and minority class is large enough.  

## We declare "keys" variable (loan ids of the downsampled data), which we will use to filter the sub tables on later to lower memory usage.

In [4]:
df_majority=APPLICATION_TRAIN[APPLICATION_TRAIN['TARGET']==0]
df_minority=APPLICATION_TRAIN[APPLICATION_TRAIN['TARGET']==1]

df_majority_downsampled = resample(df_majority,replace=False,n_samples=len(df_minority),random_state=seed) 

APPLICATION_TRAIN = pd.concat([df_majority_downsampled, df_minority]).reset_index(drop=True)

keys=APPLICATION_TRAIN.SK_ID_CURR

APPLICATION_TRAIN=APPLICATION_TRAIN.set_index('SK_ID_CURR')

del df_majority,df_minority,df_majority_downsampled; gc.collect()

21

## "generate_interactions" - function to create all pairwise combinations of *,/,-,+ for all integer and float datatype columns.  

## "correlated_columns" - function that returns all column names in a dataframe to drop that will result in no 2 columns having >.95 pearson correlation with each other. Column with more NAs is dropped.

## "aggregate_category_freq" - function that aggregates categorical variables onto a main table, creating dummy variables with each column value being the proportion of occurence for the unique id. 

## These functions are necessary to declare early on as many correlated interaction terms  are created, and identifying early on and dropping is essential for memory usage.  

## The categorical variables were not important at all in the model, using aggregates such as the mode.  This function instead computes the proportion of occurence for the unique ID, and improved the model much more.

In [5]:
def generate_interactions(df,colstart):
    column_names=df[df.columns[colstart:]].select_dtypes(include=[np.int64,np.float64]).columns
    subcolumns_exclude=[]
    for col in column_names:
        subcolumns_exclude.append(col)
        for subcol in column_names:
            if subcol not in subcolumns_exclude:
                df[col+'*'+subcol]=df[col]*df[subcol]
                df[col+'/'+subcol]=df[col]/df[subcol]
                df[col+'+'+subcol]=df[col]+df[subcol]
                df[col+'-'+subcol]=df[col]-df[subcol]
    return df

def correlated_columns(df,threshold):

    corr_matrix = df.corr().abs()

    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    print('There are %d total columns.' % (len(df.columns)))
    print('There are %d columns to remove.' % (len(to_drop)))
    
    return to_drop

def aggregate_category_freq(df,df_main,df_name,joinID):

    for col in df.select_dtypes(include=[object]).columns:
        for cat in df[col].drop_duplicates().dropna():
            catcol_aggs=pd.DataFrame((df[df[col]==cat].groupby(joinID)[col].agg('count')/df.groupby(joinID)[col].agg('count')).fillna(0))
            catcol_aggs.columns=[df_name+'.'+col+'_'+cat]
            df_main=df_main.join(catcol_aggs[df_name+'.'+col+'_'+cat],how='left')
        
    return df_main

## Declare the entityset using featuretools package.  This is how all the subtables and main table will be combined to interact with one another when creating aggregate features.

## Variables coded as numbers, specifically here "0,1", needs to be specified as categorical so the correct aggregate columns are created.  For example, we should take the mode (maybe "1", instead of the mean (maybe .712) of a binary column of 0s and 1s.

In [8]:
es = ft.EntitySet('loan_data')

## Read previous application dataset, filter, replace bad data ("365243" is an extreme outlier in all of these columns, concluding that this is probably how one of the data sources defines their NA fields)

## Use functions declared earlier to create interaction terms and drop intercorrelated ones.

In [9]:
PREVIOUS_APPLICATION = pd.read_csv("previous_application.csv",na_values=['XNA','XAP'])
PREVIOUS_APPLICATION=PREVIOUS_APPLICATION[PREVIOUS_APPLICATION.SK_ID_CURR.isin(keys)]
col_count=len(PREVIOUS_APPLICATION.columns)

for col in ['DAYS_LAST_DUE','DAYS_TERMINATION','DAYS_FIRST_DRAWING','DAYS_FIRST_DUE','DAYS_LAST_DUE_1ST_VERSION']:
    PREVIOUS_APPLICATION[col].replace(365243, np.nan, inplace=True)

PREVIOUS_APPLICATION=pd.concat([PREVIOUS_APPLICATION[PREVIOUS_APPLICATION.columns[:2]],
                                PREVIOUS_APPLICATION[PREVIOUS_APPLICATION[PREVIOUS_APPLICATION.columns[2:]].count().sort_values(ascending=False).index]],axis=1)

In [10]:
PREVIOUS_APPLICATION=generate_interactions(PREVIOUS_APPLICATION,2)
PREVIOUS_APPLICATION.dropna(how='all',axis=1,inplace=True)
PREVIOUS_APPLICATION=pd.concat([PREVIOUS_APPLICATION[PREVIOUS_APPLICATION.columns[:col_count]],
                                PREVIOUS_APPLICATION[PREVIOUS_APPLICATION[PREVIOUS_APPLICATION.columns[col_count:]].count().sort_values(ascending=False).index]],axis=1)

In [11]:
PREVIOUS_APPLICATION.drop(correlated_columns(PREVIOUS_APPLICATION,.95),axis=1,inplace=True)
APPLICATION_TRAIN=aggregate_category_freq(PREVIOUS_APPLICATION,APPLICATION_TRAIN,'PREVIOUS_APPLICATION','SK_ID_CURR')
PREVIOUS_APPLICATION.drop(PREVIOUS_APPLICATION.select_dtypes(include=[object]).columns,axis=1,inplace=True)

There are 709 total columns.
There are 524 columns to remove.


## Add previous application dataset to entityset

In [12]:
es.entity_from_dataframe(
    entity_id = 'PREVIOUS_APPLICATION', 
    dataframe = PREVIOUS_APPLICATION,
    index = 'SK_ID_PREV',
)

del PREVIOUS_APPLICATION; gc.collect()

371

## This process is repetitive, same process is done for all sub tables

In [13]:
POS_CASH_BALANCE = pd.read_csv("POS_CASH_balance.csv")
POS_CASH_BALANCE=POS_CASH_BALANCE[POS_CASH_BALANCE.SK_ID_CURR.isin(keys)]
col_count=len(POS_CASH_BALANCE.columns)
POS_CASH_BALANCE=pd.concat([POS_CASH_BALANCE[POS_CASH_BALANCE.columns[:2]],
                            POS_CASH_BALANCE[POS_CASH_BALANCE[POS_CASH_BALANCE.columns[2:]].count().sort_values(ascending=False).index]],axis=1)

In [14]:
POS_CASH_BALANCE=generate_interactions(POS_CASH_BALANCE,2)
POS_CASH_BALANCE.dropna(how='all',axis=1,inplace=True)
POS_CASH_BALANCE=pd.concat([POS_CASH_BALANCE[POS_CASH_BALANCE.columns[:col_count]],
                            POS_CASH_BALANCE[POS_CASH_BALANCE[POS_CASH_BALANCE.columns[col_count:]].count().sort_values(ascending=False).index]],axis=1)

In [15]:
POS_CASH_BALANCE.drop(correlated_columns(POS_CASH_BALANCE,.95),axis=1,inplace=True)
APPLICATION_TRAIN=aggregate_category_freq(POS_CASH_BALANCE,APPLICATION_TRAIN,'POS_CASH_BALANCE','SK_ID_CURR')
POS_CASH_BALANCE.drop(POS_CASH_BALANCE.select_dtypes(include=[object]).columns,axis=1,inplace=True)

There are 48 total columns.
There are 18 columns to remove.


In [16]:
es.entity_from_dataframe(
    entity_id = 'POS_CASH_BALANCE', 
    dataframe = POS_CASH_BALANCE,
    make_index = True,
    index='index',
)

del POS_CASH_BALANCE; gc.collect()

210

In [17]:
CREDIT_CARD_BALANCE = pd.read_csv("credit_card_balance.csv")
CREDIT_CARD_BALANCE=CREDIT_CARD_BALANCE[CREDIT_CARD_BALANCE.SK_ID_CURR.isin(keys)]
col_count=len(CREDIT_CARD_BALANCE.columns)
CREDIT_CARD_BALANCE=pd.concat([CREDIT_CARD_BALANCE[CREDIT_CARD_BALANCE.columns[:2]],
                               CREDIT_CARD_BALANCE[CREDIT_CARD_BALANCE[CREDIT_CARD_BALANCE.columns[2:]].count().sort_values(ascending=False).index]],axis=1)

In [18]:
CREDIT_CARD_BALANCE=generate_interactions(CREDIT_CARD_BALANCE,2)
CREDIT_CARD_BALANCE.dropna(how='all',axis=1,inplace=True)
#CREDIT_CARD_BALANCE=pd.concat([CREDIT_CARD_BALANCE[CREDIT_CARD_BALANCE.columns[:col_count]],
#                               CREDIT_CARD_BALANCE[CREDIT_CARD_BALANCE[CREDIT_CARD_BALANCE.columns[col_count:]].count().sort_values(ascending=False).index]],axis=1)

In [19]:
CREDIT_CARD_BALANCE.drop(correlated_columns(CREDIT_CARD_BALANCE,.95),axis=1,inplace=True)
APPLICATION_TRAIN=aggregate_category_freq(CREDIT_CARD_BALANCE,APPLICATION_TRAIN,'CREDIT_CARD_BALANCE','SK_ID_CURR')
CREDIT_CARD_BALANCE.drop(CREDIT_CARD_BALANCE.select_dtypes(include=[object]).columns,axis=1,inplace=True)

There are 783 total columns.
There are 517 columns to remove.


In [20]:
es.entity_from_dataframe(
    entity_id = 'CREDIT_CARD_BALANCE', 
    dataframe = CREDIT_CARD_BALANCE,
    make_index = True,
    index='index',
)

del CREDIT_CARD_BALANCE; gc.collect()

112

In [21]:
INSTALLMENTS_PAYMENTS = pd.read_csv("installments_payments.csv")
INSTALLMENTS_PAYMENTS=INSTALLMENTS_PAYMENTS[INSTALLMENTS_PAYMENTS.SK_ID_CURR.isin(keys)]
col_count=len(INSTALLMENTS_PAYMENTS.columns)
INSTALLMENTS_PAYMENTS=pd.concat([INSTALLMENTS_PAYMENTS[INSTALLMENTS_PAYMENTS.columns[:2]],
                               INSTALLMENTS_PAYMENTS[INSTALLMENTS_PAYMENTS[INSTALLMENTS_PAYMENTS.columns[2:]].count().sort_values(ascending=False).index]],axis=1)

In [22]:
INSTALLMENTS_PAYMENTS=generate_interactions(INSTALLMENTS_PAYMENTS,2)
INSTALLMENTS_PAYMENTS.dropna(how='all',axis=1,inplace=True)
INSTALLMENTS_PAYMENTS=pd.concat([INSTALLMENTS_PAYMENTS[INSTALLMENTS_PAYMENTS.columns[:col_count]],
                                 INSTALLMENTS_PAYMENTS[INSTALLMENTS_PAYMENTS[INSTALLMENTS_PAYMENTS.columns[col_count:]].count().sort_values(ascending=False).index]],axis=1)

In [23]:
INSTALLMENTS_PAYMENTS.drop(correlated_columns(INSTALLMENTS_PAYMENTS,.95),axis=1,inplace=True)
APPLICATION_TRAIN=aggregate_category_freq(INSTALLMENTS_PAYMENTS,APPLICATION_TRAIN,'INSTALLMENTS_PAYMENTS','SK_ID_CURR')
INSTALLMENTS_PAYMENTS.drop(INSTALLMENTS_PAYMENTS.select_dtypes(include=[object]).columns,axis=1,inplace=True)

There are 68 total columns.
There are 39 columns to remove.


In [25]:
es.entity_from_dataframe(
    entity_id = 'INSTALLMENTS_PAYMENTS', 
    dataframe = INSTALLMENTS_PAYMENTS,
    make_index = True,
    index='index',
)

del INSTALLMENTS_PAYMENTS; gc.collect()

49

In [26]:
BUREAU=pd.read_csv("bureau.csv",na_values=['XNA','XAP'])
BUREAU=BUREAU[BUREAU.SK_ID_CURR.isin(keys)]
bureaukeys=BUREAU.SK_ID_BUREAU
col_count=len(BUREAU.columns)
BUREAU=pd.concat([BUREAU[BUREAU.columns[:2]],
                  BUREAU[BUREAU[BUREAU.columns[2:]].count().sort_values(ascending=False).index]],axis=1)

In [27]:
BUREAU=generate_interactions(BUREAU,2)
BUREAU.dropna(how='all',axis=1,inplace=True)
BUREAUS=pd.concat([BUREAU[BUREAU.columns[:col_count]],
                   BUREAU[BUREAU[BUREAU.columns[col_count:]].count().sort_values(ascending=False).index]],axis=1)
BUREAU=BUREAU.set_index('SK_ID_BUREAU')

In [29]:
BUREAU.drop(correlated_columns(BUREAU,.95),axis=1,inplace=True)
APPLICATION_TRAIN=aggregate_category_freq(BUREAU,APPLICATION_TRAIN,'BUREAU','SK_ID_CURR')
BUREAU.drop(BUREAU.select_dtypes(include=[object]).columns,axis=1,inplace=True)

There are 280 total columns.
There are 153 columns to remove.


In [30]:
BUREAU_BALANCE = pd.read_csv("bureau_balance.csv",na_values=['X'])
BUREAU_BALANCE=BUREAU_BALANCE[BUREAU_BALANCE.SK_ID_BUREAU.isin(bureaukeys)]

In [31]:
BUREAU=aggregate_category_freq(BUREAU_BALANCE,BUREAU,'BUREAU_BALANCE','SK_ID_BUREAU')
BUREAU_BALANCE.drop(BUREAU_BALANCE.select_dtypes(include=[object]).columns,axis=1,inplace=True)

## BUREAU_BALANCE table contains the status for each month in DPD. 

## Many of the statuses were blank (Status of "X"), so flash fill method was used to fill in the previous month's status of the of loan for each NA.  The model performed better leaving these as NA, so we abandoned this strategy.  

# The status column is categorical, we tried converting this to numeric by replacing "C" (completed) status by 0, but leaving as categorical was best for our model.

In [32]:
BUREAU.reset_index(level=0,inplace=True)
es.entity_from_dataframe(
    entity_id = 'BUREAU', 
    dataframe = BUREAU,
    index = 'SK_ID_BUREAU',
)


del BUREAU; gc.collect()


es.entity_from_dataframe(
    entity_id = 'BUREAU_BALANCE', 
    dataframe = BUREAU_BALANCE,
    make_index=True,
    index='index',
)


del BUREAU_BALANCE; gc.collect()

28

In [33]:
APPLICATION_TRAIN.reset_index(level=0,inplace=True)
es.entity_from_dataframe(
    entity_id='APPLICATION_TRAIN',
    dataframe=APPLICATION_TRAIN.drop('TARGET', axis=1),
    index='SK_ID_CURR',
    variable_types={
        f: ft.variable_types.Categorical 
        for f in APPLICATION_TRAIN.columns if f.startswith('FLAG_')
    }
)

del APPLICATION_TRAIN; gc.collect()


relationship1 = ft.Relationship(
    es['APPLICATION_TRAIN']['SK_ID_CURR'],
    es['PREVIOUS_APPLICATION']['SK_ID_CURR']
)

relationship2 = ft.Relationship(
    es['APPLICATION_TRAIN']['SK_ID_CURR'],
    es['BUREAU']['SK_ID_CURR']
)

relationship3 = ft.Relationship(
    es['APPLICATION_TRAIN']['SK_ID_CURR'],
    es['POS_CASH_BALANCE']['SK_ID_CURR']
)

relationship4 = ft.Relationship(
    es['APPLICATION_TRAIN']['SK_ID_CURR'],
    es['CREDIT_CARD_BALANCE']['SK_ID_CURR']
)


relationship5 = ft.Relationship(
    es['APPLICATION_TRAIN']['SK_ID_CURR'],
    es['INSTALLMENTS_PAYMENTS']['SK_ID_CURR']
)

relationship6 = ft.Relationship(
    es['BUREAU']['SK_ID_BUREAU'],
    es['BUREAU_BALANCE']['SK_ID_BUREAU']
)

es.add_relationship(relationship1)
es.add_relationship(relationship2)
es.add_relationship(relationship3)
es.add_relationship(relationship4)
es.add_relationship(relationship5)
es.add_relationship(relationship6)

Entityset: loan_data
  Entities:
    PREVIOUS_APPLICATION [Rows: 234942, Columns: 169]
    POS_CASH_BALANCE [Rows: 1321544, Columns: 30]
    CREDIT_CARD_BALANCE [Rows: 488676, Columns: 266]
    INSTALLMENTS_PAYMENTS [Rows: 1813984, Columns: 30]
    BUREAU [Rows: 232790, Columns: 132]
    BUREAU_BALANCE [Rows: 2202742, Columns: 3]
    APPLICATION_TRAIN [Rows: 49650, Columns: 286]
  Relationships:
    PREVIOUS_APPLICATION.SK_ID_CURR -> APPLICATION_TRAIN.SK_ID_CURR
    BUREAU.SK_ID_CURR -> APPLICATION_TRAIN.SK_ID_CURR
    POS_CASH_BALANCE.SK_ID_CURR -> APPLICATION_TRAIN.SK_ID_CURR
    CREDIT_CARD_BALANCE.SK_ID_CURR -> APPLICATION_TRAIN.SK_ID_CURR
    INSTALLMENTS_PAYMENTS.SK_ID_CURR -> APPLICATION_TRAIN.SK_ID_CURR
    BUREAU_BALANCE.SK_ID_BUREAU -> BUREAU.SK_ID_BUREAU

## Below we are creating the feature matrix (fm) that adds in aggregate features defined below for all sub tables.  

## For all numeric features, we'll add the instance count, mean, median, skew, sum, standard deviation, max, and min.  

## categorical features were converted to numeric proportions (explained eariler).

In [34]:
fm, feature_defs = ft.dfs(
    entityset=es, 
    target_entity="APPLICATION_TRAIN", 
    agg_primitives=[
        #"avg_time_between",
        #"time_since_last",
        "mean",
        "median",
        #"num_unique",
        "count",
        "skew",
        "sum",
        "std",
        #"mode",
        "max",
        "min"
    ],
    trans_primitives=[
        #"time_since_previous", 
        #"cum_mean",
        #"cum_max",
        #"cum_min"
        #"percentile",
        #"add"
    ],
    max_depth=2,
    #cutoff_time=cutoff_times,
    training_window=ft.Timedelta(60, "d"),
    max_features=5000,
    chunk_size=4000,
    verbose=True,
)

Built 4667 features
Elapsed: 2:06:48 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 13/13 chunks 


## As this is a long process, will save to a .pkl file for use in the modeling notebook

In [35]:
fm.to_pickle('loan data.pkl')